<a href="https://colab.research.google.com/github/shawna114/big-data-challenge/blob/main/Copy_of_level_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [75]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.1'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()


Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:4 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:5 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Ign:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:10 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graph

In [76]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-03-20 00:38:28--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.1’

postgresql-42.2.9.j 100%[===================>] 892.61K  5.09MB/s    in 0.2s    

2022-03-20 00:38:28 (5.09 MB/s) - ‘postgresql-42.2.9.jar.1’ saved [914037/914037]



In [114]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()


In [111]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url ="https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Personal_Care_Appliances_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Personal_Care_Appliances_v1_00.tsv.gz"), header=True, inferSchema=True, sep='\t')

# Show DataFrame
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   32114233|R1QX6706ZWJ1P5|B00OYRW4UE|     223980852|Elite Sportz Exer...|Personal_Care_App...|          5|            0|          0|   N|                Y|Good quality. Shi...|Exactly as descri...| 2015-08-31|
|         US|   18125776|R3QWMLJHIW6P37|B0000537JQ|     819771537|     Ezy Dose Weekly|Personal_Care_App

In [79]:
from pyspark.sql.functions import to_date

In [80]:
df.count()

85981

In [81]:
df.describe()

DataFrame[summary: string, marketplace: string, customer_id: string, review_id: string, product_id: string, product_parent: string, product_title: string, product_category: string, star_rating: string, helpful_votes: string, total_votes: string, vine: string, verified_purchase: string, review_headline: string, review_body: string, review_date: string]

In [82]:
# Drop NaN from dataset
df = df.dropna()

In [83]:
# Drop duplicates from dataset
df=df.distinct()

In [84]:
df.count()

85978

In [100]:

review_id_table = df.select(["review_id", "customer_id", "product_id","product_parent", to_date("review_date", "yyyy-MM-dd").alias("review_date")])
review_id_table.show(5)

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R247N8Z8Q5YDX1|    1734409|B0001UMQ5Q|     943235804| 2015-08-31|
|R3QTPVKL2HKQR6|   27215351|B00J7G8EL0|     381088677| 2015-08-27|
| RXYIQ0T8QIXLP|   16499933|B00V2L5JRA|     417635592| 2015-08-26|
| R3RAHZC5LHL5X|   17554085|B00HES9CMS|     318730927| 2015-08-26|
| RYXTV7YBMJLH1|    5177140|B00XZJ2G46|     217870256| 2015-08-25|
+--------------+-----------+----------+--------------+-----------+
only showing top 5 rows



In [99]:

products = df.select(["product_id","product_title"]).distinct()
products.show(5)

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00J7G8EL0|GUM Expanding Flo...|
|B003BIFRFQ|Pjur Backdoor Rel...|
|B000KE8ASK|Inc Magnetic Peda...|
|B00075VVYC|Reizen RL-350 Bra...|
|B009AZQSF2|Moon Calendar 201...|
+----------+--------------------+
only showing top 5 rows



In [96]:
customers = df.groupby("customer_id").agg({"customer_id": "count"}).withColumnRenamed("count(customer_id)", "customer_count")
customers.show(5)

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   16800802|             1|
|   32976373|             1|
|   21635400|             1|
|   17503394|             1|
|   16855262|             1|
+-----------+--------------+
only showing top 5 rows



In [97]:
vine_table = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])
vine_table.show(5)

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R247N8Z8Q5YDX1|          4|            0|          0|   N|
|R3QTPVKL2HKQR6|          4|            1|          1|   N|
| RXYIQ0T8QIXLP|          3|            1|          2|   N|
| R3RAHZC5LHL5X|          5|            0|          0|   N|
| RYXTV7YBMJLH1|          5|            0|          0|   N|
+--------------+-----------+-------------+-----------+----+
only showing top 5 rows



In [243]:
 # Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://mypostgresdb.cnnfcbfbmueb.us-west-2.rds.amazonaws.com:5432/shawna_db"
config = {"user":"shawnactruitt", 
          "password": "qazWSXqazWSX",
          "driver": "org.postgresql.Driver"}

In [245]:
# Write review_id_df to table in RDS
review_id_table.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

Py4JJavaError: ignored

In [172]:
# Write review_id_df to table in RDS
products.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

Py4JJavaError: ignored

In [69]:
# Write review_id_df to table in RDS
customers.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

AttributeError: ignored

In [95]:

vine_table.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)

IllegalArgumentException: ignored

In [ ]:
# Write review_id_df to table in RDS
customers.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

AttributeError: ignored